# Classifying Movie Reviews from IMDb
### by Marty VanHoof

The Internet Movie Database (IMDb) is an online database consisting of movie, TV, and video game information.  Our goal is to analyze a dataset of movie reviews from IMDb and use the Python neural network library [Keras](https://keras.io/) to predict the sentiment of a movie review.  The dataset consists of 25,000 movie reviews from IMDb, labeled by sentiment (positive review or negative review). Reviews have been preprocessed, and each review is encoded as a sequence of word indices (integers).  Words are indexed by overall frequency in the dataset, so for example the integer 1 corresponds to the most frequent word in the dataset, 2 corresponds to the 2nd most frequent word, etc.  Therefore, a sentence is represented by a sequence of integers, and each movie review can be encoded as an integer vector.

For humans, it is relatively easy to predict the sentiment of a review.  This is something we want to train the model to do.

In [1]:
import numpy as np
import pandas as pd
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# fix random seed for reproducibility
np.random.seed(13)

Using TensorFlow backend.


## Load and prepare the data

With Keras, this dataset comes [preloaded](https://keras.io/datasets/), so a simple command will allow us to access the training and testing data. The parameter **`num_words`** specifies the top most frequent words to consider.  The training and testing data are loaded into matrices **`X_train`** and **`X_test`** (represented as numpy arrays), and the corresponding training/testing labels (**`y_train/y_test`**) are vectors of binary integers (0 or 1), where 0 represents a bad review and 1 represents a good review.

In [2]:
# Load the data and split it into 50% train, 50% test sets
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=5000)

print('number of training examples: ', len(X_train))
print('number of testing examples: ', len(X_test))
print('classes: ', np.unique(y_train))

number of training examples:  25000
number of testing examples:  25000
classes:  [0 1]


We can look at the first training example, which is a movie review encoded as an array of integers

In [3]:
print(X_train[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 2, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 2, 19, 178, 32]


## How to recover the orginal review?

The above array is how the algorithm will see the review, but it would be nice to see the review in human-readable form as well, so we can get an idea of the review's sentiment.  There is a function in Keras called **`get_word_index`** that returns a dictionary mapping words to their corresponding indices.  For example, we can have a look at (say, 10) random entries from the dictionary with the restriction that these 10 entries come from the top 5000 most frequent words.

In [4]:
# print out 10 random entries from the top 5000 most frequent words in the word_index dict
word_index = imdb.get_word_index()
rand_indices = np.random.randint(5000, size=10)
for k,v in word_index.items():
    if v in rand_indices:
        print(k,v)

wont 4280
roger 2474
coherent 3981
doctor 1039
under 464
sensitive 2725
each 254
floor 1870
ken 3659
sally 3507


We can use this word index and write a function that translates an integer-encoded review back into human-readable form.  For more information, see the first answer to this [stackoverflow post](https://stackoverflow.com/questions/42821330/restore-original-text-from-keras-s-imdb-dataset).  

In [5]:
def get_review(review_number):
    '''
    Put the review back in a form that humans can understand
    '''
    word_index = imdb.get_word_index()
    word_index = {k:v+3 for k, v in word_index.items()}
    word_index["<PAD>"] = 0
    word_index["<START>"] = 1
    word_index["<UNK>"] = 2
    index_word = {v:k for k, v in word_index.items()}
    return ' '.join( index_word[i] for i in X_train[review_number] )

# print out the first two reviews and their labels
print('The first review is clearly positive, so it should get a label of {} :\n'.format(y_train[0]))
print(get_review(0), '\n')
print('The second review is clearly negative, so it should get a label of {} :\n'.format(y_train[1]))
print(get_review(1))

The first review is clearly positive, so it should get a label of 1 :

<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly <UNK> was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little <UNK> that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big <UNK> for the whol

## Sequence padding

The neural network model requires that the input vectors all have the same length, so we need to set a fixed length for the inputs and then find a way to deal with the movie reviews that are too long or too short.  This can be done using [sequence.pad_sequences](https://keras.io/preprocessing/sequence/) in the **`keras.preprocessing`** module, which will truncate longer reviews to a fixed length and also pad shorter reviews with zeros.

In [3]:
X_train = sequence.pad_sequences(X_train, maxlen=500)
X_test = sequence.pad_sequences(X_test, maxlen=500)

print('train/test set dimensions: ', X_train.shape, X_test.shape, '\n')
print('first sentence in training set, encoded and padded:\n\n ', X_train[0], '\n')

train/test set dimensions:  (25000, 500) (25000, 500) 

first sentence in training set, encoded and padded:

  [   0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0   

## Using GridSearchCV to find optimal hyperparameters

First I want to give a shout-out to Dr. Jason Brownlee, who's [article on grid-searching with Keras](https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/) is immensely helpful.

[GridSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV) is a very useful class in scikit-learn that performs an exhaustive search over specified hyperparameters of the model and finds those with the most optimal performance.  It saves a lot of time, since now we don't have to tune the hyperparameters manually.  Keras does not have grid search capabilities built into it, but it does have a [scikit-learn wrapper](https://keras.io/scikit-learn-api/) that we can use to interface with the scikit-learn API.  An important thing to note is that we are performing the grid search by tuning a few different hyperparameters separately and then aggregating the results.  This is not the best way to grid search because there may be dependence relationships among the hyperparameters.  There are two reasons I'm doing it this way:  1)  It's too time (and CPU) consuming to tune many hyperparameters at once on my little laptop; 2) tuning the hyperparameters this way the first time is more illustrative and better for my learning process.

First we build and compile the model in Keras using the **`build_model`** function below.  Then we can pass this function as an argument to **`KerasClassifier`** in order to interface with scikit-learn and use **`GridSearchCV`**.
The **`grid_search`** function below takes the sklearn-wrapped Keras model and a parameter grid as arguments and then performs the grid search.  It returns a dictionary with some useful summary statistics of the grid combinations and it also prints the accuracy score and the hyperparameters of the optimal model.  We will eventually load the dictionary into a Pandas dataframe to display the results in a nice way.  Most of these steps are implemented in the **`main`** function in the [imdb_models.py](https://github.com/marty-vanhoof/IMdb_Movie_Review_Classifier/blob/master/imdb_models.py) file.  The **`main`** function will also save the weights from the best model to a `.hdf5` file, and it will write the grid search results to a `.csv` file.  

It's not a good idea to try to train neural network models in a Jupyter Notebook as the process tends to be very slow and will often just crash.  In fact, it's not a great idea to try to train neural networks that are too big on a laptop CPU either.  I'm going to eventually look into some form of cloud computing option, such as [Amazon Web Services](https://aws.amazon.com/).

The model is a sequential model, which is just a linear stack of layers in the neural network.  The first layer is an [Embedding](https://keras.io/layers/embeddings/) layer that maps each integer-encoded word into a space of word vectors where semantically similar words are mapped to nearby points.  This natural language processing model is called [word2vec](https://www.tensorflow.org/tutorials/word2vec).

In [4]:
def build_model(optimizer='adagrad', learn_rate=0.01, learn_rate_decay=0.0, activation='relu',
                input_dim=5000, output_dim=32, max_review_length=500):
    '''
    Setup the model architecture and compile the model.
    '''
    
    model = Sequential()

    # embedding layer
    model.add(Embedding(input_dim, output_dim, input_length=max_review_length))
    model.add(Flatten())
    
    # first hidden layer 
    model.add(Dense(250, activation=activation))
    model.add(Dropout(0.2))
    
    # second hidden layer
    model.add(Dense(250, activation=activation))
    model.add(Dropout(0.2))
    
    # output layer
    model.add(Dense(1, activation='sigmoid'))
    
    # compile the model
    optimizer = Adagrad(lr=learn_rate, decay=learn_rate_decay)
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
    
    return model

def grid_search(model, param_grid):
    '''
    Perform a grid search of model hyperparameters and returns the best model,
    along with some summary statistics of the different grid combinations.
    '''
    
    # set up and perform the grid search
    grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
    grid_result = grid.fit(X_train, y_train)

    # the cv_results_ attribute is a dict that summarizes many important results
    # from each model
    test_score_means = grid_result.cv_results_['mean_test_score']
    train_score_means = grid_result.cv_results_['mean_train_score']
    train_times = grid_result.cv_results_['mean_fit_time']
    params = grid_result.cv_results_['params']

    # store these results in a smaller dict for easy loading into a pandas dataframe
    final_results = dict(mean_test_score=test_score_means, mean_train_score=train_score_means,
                         mean_fit_time=train_times, params=params)

    print('Best score {} using hyperparameters {}'.format(grid_result.best_score_,
                                                          grid_result.best_params_))
    
    return grid_result, final_results

m = build_model()
m.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
flatten_1 (Flatten)          (None, 16000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 250)               4000250   
_________________________________________________________________
dropout_1 (Dropout)          (None, 250)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 250)               62750     
_________________________________________________________________
dropout_2 (Dropout)          (None, 250)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 251       
Total para

## 1. Tune batch size and number of epochs (and two optimizers)

**Best score 0.86724 using hyperparameters {'optimizer': 'adam', 'batch_size': 200, 'epochs': 2}**

We ran the grid search in the **`imdb_models.py`** file with the following parameter grid:

```
optimizer = ['rmsprop', 'adam']
batch_size = [100, 200, 500]
epochs = [2, 5, 10]
```

The function below will display some summary statistics of the different grid combinations, ordered by mean_test_score in descending order.  Only the first 10 results are shown.  

In [5]:
def display_results(csv_file, num_rows=0):
    '''
    Display the model results in a Pandas dataframe
    '''
    from IPython.display import display
    
    # load the results from the csv file, sort the values, and reset the index
    results = pd.read_csv(csv_file)
    results.sort_values(by=['mean_test_score'], inplace=True, ascending=False)
    results.reset_index(drop=True, inplace=True)
    
    # set the last column to display with the maximum width
    pd.set_option('display.max_colwidth', -1)
    
    # convert mean_fit_time to minutes
    pd.to_numeric(results.mean_fit_time)
    results['mean_fit_time'] = results.mean_fit_time / 60
    
    print('\nTotal training time: {} minutes'.format(round(np.sum(results.mean_fit_time), 2)))
    
    if num_rows:
        display(results.head(num_rows))
    else:
        display(results)
    
display_results('imdb_results/grid_batch_epoch_results.csv', 10)


Total training time: 83.01 minutes


,mean_fit_time,mean_test_score,mean_train_score,params
0,1.573460,0.86724,0.97924,"{'optimizer': 'adam', 'batch_size': 200, 'epochs': 2}"
1,1.627119,0.86104,0.98642,"{'optimizer': 'rmsprop', 'batch_size': 100, 'epochs': 2}"
2,1.803518,0.85900,0.99240,"{'optimizer': 'adam', 'batch_size': 100, 'epochs': 2}"
3,4.999639,0.85636,1.00000,"{'optimizer': 'adam', 'batch_size': 500, 'epochs': 10}"
4,7.485352,0.85552,1.00000,"{'optimizer': 'adam', 'batch_size': 200, 'epochs': 10}"
5,1.476564,0.85532,0.95674,"{'optimizer': 'rmsprop', 'batch_size': 200, 'epochs': 2}"
6,3.653290,0.85204,0.99998,"{'optimizer': 'adam', 'batch_size': 200, 'epochs': 5}"
7,4.410074,0.85148,0.99998,"{'optimizer': 'adam', 'batch_size': 100, 'epochs': 5}"
8,8.487769,0.85136,0.99996,"{'optimizer': 'adam', 'batch_size': 100, 'epochs': 10}"
9,6.102054,0.85060,0.99998,"{'optimizer': 'rmsprop', 'batch_size': 500, 'epochs': 10}"


### Check accuracy on the training and test sets

We can see that the training accuracy is quite a bit higher than the accuracy on the test set.  This means that the model has overfit, and this is only after 2 epochs.

In [10]:
def check_model(hdf5_file):
    '''
    Load the weights from the grid search and check the accuracy
    on the test set.
    '''
    from imdb_models import build_model

    model = build_model()

    # load the weights that yielded the best validation score from grid search
    model.load_weights(hdf5_file)

    # evaluate test accuracy
    train_score = model.evaluate(X_train, y_train, verbose=0)
    test_score = model.evaluate(X_test, y_test, verbose=0)
    train_accuracy = 100*train_score[1]
    test_accuracy = 100*test_score[1]
    print('\nTraining accuracy: {}%'.format(round(train_accuracy, 2)))
    print('Test set accuracy: {}%'.format(round(test_accuracy, 2)))
    
check_model('imdb_results/imdb_batch_epoch_best.hdf5')


Training accuracy: 98.02%
Test set accuracy: 86.43%


## 2. Tune the training optimization algorithm

**Best score 0.87756 using hyperparameters {'batch_size': 200, 'epochs': 2, 'optimizer': 'adagrad'}**

Letting the optimizers vary and choosing the batch size and number of epochs based on the results from 1, we used the following parameter grid:

```
optimizer = ['adam', 'rmsprop', 'sgd', 'adagrad', 'adadelta', 'adamax', 'nadam']
batch_size = [200]
epochs = [2]
```

In [11]:
display_results('imdb_results/grid_optimizer_results.csv')


Total training time: 9.69 minutes


,mean_fit_time,mean_test_score,mean_train_score,params
0,1.322234,0.87756,0.970920,"{'batch_size': 200, 'epochs': 2, 'optimizer': 'adagrad'}"
1,1.460754,0.86720,0.981940,"{'batch_size': 200, 'epochs': 2, 'optimizer': 'adam'}"
2,1.278613,0.86424,0.989620,"{'batch_size': 200, 'epochs': 2, 'optimizer': 'nadam'}"
3,1.346801,0.86328,0.969620,"{'batch_size': 200, 'epochs': 2, 'optimizer': 'rmsprop'}"
4,1.458367,0.84208,0.922919,"{'batch_size': 200, 'epochs': 2, 'optimizer': 'adamax'}"
5,1.614987,0.55312,0.581459,"{'batch_size': 200, 'epochs': 2, 'optimizer': 'adadelta'}"
6,1.207428,0.50816,0.513540,"{'batch_size': 200, 'epochs': 2, 'optimizer': 'sgd'}"


### Check accuracy on the training and test sets

In [12]:
check_model('imdb_results/imdb_optimizer_best.hdf5')


Training accuracy: 96.91%
Test set accuracy: 87.98%


## 3. Tune learn rate and learn rate decay for adagrad

**Best score 0.87204 using hyperparameters {'learn_rate_decay': 0.01, 'learn_rate': 0.01}**

This time we used adagrad (along with the same setting for batch size and epochs from above), and the following parameter grid:

```
learn_rate = [0.001, 0.01, 0.1, 0.3]
learn_rate_decay = [0.0, 0.1, 0.01, 0.001]
```

In [15]:
display_results('imdb_results/grid_learn_rate_results.csv', 10)


Total training time: 21.8 minutes


,mean_fit_time,mean_test_score,mean_train_score,params
0,1.375478,0.87204,0.943220,"{'learn_rate_decay': 0.01, 'learn_rate': 0.01}"
1,1.481537,0.86612,0.958160,"{'learn_rate_decay': 0.0, 'learn_rate': 0.01}"
2,1.423921,0.86516,0.949180,"{'learn_rate_decay': 0.001, 'learn_rate': 0.01}"
3,1.351528,0.66376,0.725540,"{'learn_rate_decay': 0.0, 'learn_rate': 0.001}"
4,1.399060,0.64552,0.704220,"{'learn_rate_decay': 0.1, 'learn_rate': 0.01}"
5,1.384608,0.59864,0.656881,"{'learn_rate_decay': 0.001, 'learn_rate': 0.001}"
6,1.455008,0.58128,0.637660,"{'learn_rate_decay': 0.01, 'learn_rate': 0.001}"
7,1.449480,0.52924,0.555740,"{'learn_rate_decay': 0.1, 'learn_rate': 0.001}"
8,1.273191,0.50412,0.497940,"{'learn_rate_decay': 0.0, 'learn_rate': 0.3}"
9,1.275629,0.50344,0.498280,"{'learn_rate_decay': 0.1, 'learn_rate': 0.3}"


### Check accuracy

The accuracy is a tad higher and there is less overfitting as well.

In [16]:
check_model('imdb_results/imdb_learn_rate_best.hdf5')


Training accuracy: 94.75%
Test set accuracy: 88.22%


## 4. Tune the activation function in the hidden layers

**Best score 0.87804 using hyperparameters {'activation': 'hard_sigmoid'}**

The activations functions introduce non-linearity into the network and allow the model to learn more complicated functions.  We will try the grid search with all the different choices for the activation function in Keras.  Our neural network has 2 hidden layers, and the same activation function will be applied in both layers:

```
['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
```

In [17]:
display_results('imdb_results/grid_activation_results.csv')


Total training time: 10.85 minutes


,mean_fit_time,mean_test_score,mean_train_score,params
0,1.375843,0.87804,0.921720,{'activation': 'hard_sigmoid'}
1,1.410847,0.87656,0.953460,{'activation': 'relu'}
2,1.377378,0.86892,0.919180,{'activation': 'sigmoid'}
3,1.340045,0.86780,0.899880,{'activation': 'softplus'}
4,1.383792,0.86312,0.968280,{'activation': 'softsign'}
5,1.319819,0.85608,0.890120,{'activation': 'softmax'}
6,1.342223,0.75376,0.826056,{'activation': 'tanh'}
7,1.296360,0.72960,0.789442,{'activation': 'linear'}


### Check accuracy

It seems that this model did quite a bit worse.  I'm a bit perplexed why the results below are so different from the results using grid search.

In [19]:
check_model('imdb_results/imdb_activation_best.hdf5')


Training accuracy: 82.82%
Test set accuracy: 80.34%


### Neural Network with LSTM Layer

Recurrent neural networks and LSTMs are a pretty complicated topic in deep learning, and I'm only at the point where I barely understand these topics myself.  One of the dangers with using a high-level library like Keras (in my opinion) is that it can be way too easy to implement these models without having a clue what the hell you're doing.  In order to really understand Machine Learning (and deep learning in particular), it's important to understand the mathematics behind the algorithms and techniques.  A really nice lecture on recurrent neural networks is available [here](https://www.youtube.com/watch?v=6niqTuYFZLQ&list=PL3FW7Lu3i5JvHM8ljYj-zLfQRF3EO8sYv) (starting at about 9 minutes into the video).  It is from Stanford's [CS 231N Course](http://cs231n.stanford.edu/syllabus.html) on Convolutional Neural Networks for Image Recognition.  There is also a great online deep learning textbook available [here](http://www.deeplearningbook.org/). 

We will build a sequential model with an embedding layer and an LSTM layer. The sequential model is just a linear stack of layers in the neural network.  The first layer is an [Embedding](https://keras.io/layers/embeddings/) layer that maps each integer-encoded word into a space of word vectors where semantically similar words are mapped to nearby points.  This natural language processing model is called [word2vec](https://www.tensorflow.org/tutorials/word2vec).

After the embedding layer, we add an [LSTM](https://keras.io/layers/recurrent/#lstm) layer.  LSTM is an abbreviation for Long Short-Term Memory Unit, and these were originally proposed as a solution to the [vanishing gradient problem](https://en.wikipedia.org/wiki/Vanishing_gradient_problem) (and the exploding gradient problem).  In a nutshell, the vanishing/exploding gradient problem are problems when training neural networks with gradient descent methods and back-propagation.  When a neural network is trained, it tries to minimize a cost function by back-propagating the error through the network and repeatedly computing the gradients of composite functions through the layers.  Depending on the activation functions used, the gradients can either become so small or so large that further weight updates become infeasible.  LSTMs help preserve the error that is back-propagated through the network. 

We will start by trying the sigmoid activation function.

In [7]:
# model parameters
top_words = 5000
embedding_dim = 32
max_review_length = 500
activation_function = 'sigmoid'
loss_function = 'binary_crossentropy'
optimization_method = 'adam'
evaluation_metrics = ['accuracy']

def build_lstm_model(input_dim=top_words, output_dim=embedding_dim, input_length=max_review_length,
                activation=activation_function, loss=loss_function,
                optimizer=optimization_method, metrics=evaluation_metrics):
    '''
    Setup the model architecture, compile the model, and give a model summary.
    '''
    model = Sequential()
    model.add(Embedding(input_dim, output_dim, input_length=max_review_length))
    model.add(LSTM(100))
    model.add(Dense(1, activation=activation_function))
    model.compile(optimizer, loss, metrics=evaluation_metrics)
    print(model.summary())
    return model

imdb_model = build_model();

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 213,301.0
Trainable params: 213,301
Non-trainable params: 0.0
_________________________________________________________________
None
